In [1]:
import numpy as np

In [2]:
# Define the sequence to be aligned
sequence1 = 'GCTGG'
sequence2 = 'CACTG'

In [3]:
# Define the HMM parameters
# For simplicity, we assume a fixed length for sequences and a fixed gap penalty.

# Define the states (M: match, I: insertion, D: deletion)
states = ['M', 'I', 'D']

# Example: Transitions from M to M, M to I, M to D, etc.
# Define the transition probabilities
transition_probabilities = {
    'M': {'M': 0.7, 'I': 0.1, 'D': 0.2},
    'I': {'M': 0.3, 'I': 0.4, 'D': 0.3},
    'D': {'M': 0.2, 'I': 0.3, 'D': 0.5}
}


# Define emission probabilities (for simplicity, assume equal probabilities)
emission_probabilities = {
    'M': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    'I': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    'D': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
}

In [4]:
# Create a matrix to store the forward probabilities
num_states = len(states)
len_seq1 = len(sequence1)
len_seq2 = len(sequence2)
forward_matrix = np.zeros((num_states, len_seq1 + 1, len_seq2 + 1))

In [5]:
# Initialize the forward matrix
forward_matrix[:, 0, 0] = 1.0

# Fill in the forward matrix using the forward algorithm
for i in range(1, len_seq1 + 1):
    for j in range(1, len_seq2 + 1):
        for state1 in states:
            for state2 in states:
                # Calculate the forward probability for the current cell
                transition_prob = transition_probabilities[state1][state2]
                emission_prob_seq1 = emission_probabilities[state1].get(sequence1[i - 1], 0.0)
                emission_prob_seq2 = emission_probabilities[state2].get(sequence2[j - 1], 0.0)
                forward_matrix[num_states - 1, i, j] += forward_matrix[states.index(state1), i - 1, j - 1] * \
                    transition_prob * emission_prob_seq1 * emission_prob_seq2

In [6]:
# Calculate the total likelihood of the sequences
likelihood = forward_matrix.sum()

print(f"Likelihood of the alignment: {likelihood}")

Likelihood of the alignment: 3.1999998092651367


In [7]:
# Create a matrix to store the backward probabilities
backward_matrix = np.zeros((num_states, len_seq1 + 1, len_seq2 + 1))

In [8]:
# Initialize the backward matrix
backward_matrix[:, len_seq1, len_seq2] = 1.0

# Fill in the backward matrix using the backward algorithm
for i in range(len_seq1, 0, -1):
    for j in range(len_seq2, 0, -1):
        for state1 in states:
            for state2 in states:
                # Calculate the backward probability for the current cell
                transition_prob = transition_probabilities[state1][state2]
                emission_prob_seq1 = emission_probabilities[state1].get(sequence1[i - 1], 0.0)
                emission_prob_seq2 = emission_probabilities[state2].get(sequence2[j - 1], 0.0)
                backward_matrix[states.index(state1), i - 1, j - 1] += \
                    transition_prob * emission_prob_seq1 * emission_prob_seq2 * \
                    backward_matrix[num_states - 1, i, j]

In [9]:
# Calculate the total likelihood of the sequences from the backward matrix
total_likelihood_backward = (1 / len(states)) * (1 / len(states)) * np.sum(
    [backward_matrix[states.index(state1), 0, 0] for state1 in states])

print(f"Likelihood of the alignment (backward algorithm): {total_likelihood_backward}")

Likelihood of the alignment (backward algorithm): 3.178914388020833e-07


In [10]:
# Create a matrix to store the Viterbi path (backtrack matrix)
viterbi_matrix = np.zeros((num_states, len_seq1 + 1, len_seq2 + 1), dtype=int)
viterbi_path = []

In [11]:
# Initialize the Viterbi matrix and path
viterbi_matrix[:, 0, 0] = 1.0

# Fill in the Viterbi matrix and backtrack matrix using the Viterbi algorithm
for i in range(1, len_seq1 + 1):
    for j in range(1, len_seq2 + 1):
        for state1 in states:
            max_prob = 0.0
            max_state = None
            for state2 in states:
                # Calculate the Viterbi probability for the current cell
                transition_prob = transition_probabilities[state1][state2]
                emission_prob_seq1 = emission_probabilities[state1].get(sequence1[i - 1], 0.0)
                emission_prob_seq2 = emission_probabilities[state2].get(sequence2[j - 1], 0.0)
                viterbi_prob = viterbi_matrix[states.index(state2), i - 1, j - 1] * \
                    transition_prob * emission_prob_seq1 * emission_prob_seq2

                # Track the maximum probability and state
                if viterbi_prob > max_prob:
                    max_prob = viterbi_prob
                    max_state = state2

                # Debugging: Print intermediate results
                print(f"i={i}, j={j}, state1={state1}, state2={state2}, viterbi_prob={viterbi_prob}")

            # Update the Viterbi matrix and backtrack matrix
            viterbi_matrix[states.index(state1), i, j] = max_prob
            viterbi_path.append(max_state)

i=1, j=1, state1=M, state2=M, viterbi_prob=0.04375
i=1, j=1, state1=M, state2=I, viterbi_prob=0.00625
i=1, j=1, state1=M, state2=D, viterbi_prob=0.0125
i=1, j=1, state1=I, state2=M, viterbi_prob=0.01875
i=1, j=1, state1=I, state2=I, viterbi_prob=0.025
i=1, j=1, state1=I, state2=D, viterbi_prob=0.01875
i=1, j=1, state1=D, state2=M, viterbi_prob=0.0125
i=1, j=1, state1=D, state2=I, viterbi_prob=0.01875
i=1, j=1, state1=D, state2=D, viterbi_prob=0.03125
i=1, j=2, state1=M, state2=M, viterbi_prob=0.0
i=1, j=2, state1=M, state2=I, viterbi_prob=0.0
i=1, j=2, state1=M, state2=D, viterbi_prob=0.0
i=1, j=2, state1=I, state2=M, viterbi_prob=0.0
i=1, j=2, state1=I, state2=I, viterbi_prob=0.0
i=1, j=2, state1=I, state2=D, viterbi_prob=0.0
i=1, j=2, state1=D, state2=M, viterbi_prob=0.0
i=1, j=2, state1=D, state2=I, viterbi_prob=0.0
i=1, j=2, state1=D, state2=D, viterbi_prob=0.0
i=1, j=3, state1=M, state2=M, viterbi_prob=0.0
i=1, j=3, state1=M, state2=I, viterbi_prob=0.0
i=1, j=3, state1=M, state2=D

In [12]:
# Find the best alignment by backtracking
alignment = []
state_idx = states.index('M')
i, j = len_seq1, len_seq2

while i > 0 or j > 0:
    char_seq1 = sequence1[i - 1]
    char_seq2 = sequence2[j - 1]
    
    # Determine whether it's a match or mismatch
    if state_idx == 0 and char_seq1 == char_seq2:
        alignment.insert(0, (char_seq1, char_seq2, 'M'))
    elif state_idx == 0 and char_seq1 != char_seq2:
        alignment.insert(0, (char_seq1, char_seq2, 'X'))  # 'X' for mismatch
    elif state_idx == 1:
        alignment.insert(0, ('-', char_seq2, 'I'))  # '-' for insertion in seq1
    else:
        alignment.insert(0, (char_seq1, '-', 'D'))  # '-' for deletion in seq2
    
    if viterbi_path:
        next_state = viterbi_path.pop()
        if next_state is not None:
            state_idx = states.index(next_state)
    
    # Adjust state index for matches, insertions, and deletions
    if state_idx == 0:
        i -= 1
        j -= 1
    elif state_idx == 1:
        i -= 1
    elif state_idx == 2:
        j -= 1

# Print the best alignment
for a, b, state in alignment:
    print(f"{a} - {b} ({state})")

G - C (X)
C - A (X)
T - C (X)
G - T (X)
G - G (M)
